In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from mmctools.dataloaders import read_dir # for reading a series of data files
from mmctools.measurements import metmast # provides readers for metmast data

In [4]:
from mmctools import helper_functions as fun

# Unit tests: virtual temperature conversion
sample data from WFIP2 Physics Site PS07 met station

In [5]:
datadir = '/Users/equon/WFIP2/PS07/201611'
starttime = pd.to_datetime('2016-11-21 17:00')
endtime = pd.to_datetime('2016-11-22 04:00')

## Data I/O

In [6]:
# sensor data format
metmast.RMYoung_05106

OrderedDict([('ID', None),
             ('year', '%Y'),
             ('day', '%j'),
             ('time', '%H%M'),
             ('HorizontalWind', 1),
             ('wspd', 1),
             ('wdir', 1),
             ('wdir_std', 1),
             ('T', <function mmctools.measurements.metmast.<lambda>(Ta)>),
             ('RH', 1),
             ('P', 1),
             ('SW_down', 1),
             ('T10X', 1),
             ('p10X', 1)])

Basic usage (with generic CSV reader):

`df = read_dir(datadir)`

Met data usage:

`df = read_dir(datadir, reader=metmast.read_data, column_spec=metmast.RMYoung_05106)`

In [7]:
df = read_dir(datadir, ext='txt',
              file_filter='*.2016112[12].*', # read 20161121 and 20161122 only
              reader=metmast.read_data, column_spec=metmast.RMYoung_05106,
              height=3.0, # measurement height
              datetime_offset=-60, # shift 60s to line up with begining of interval
              start=starttime, end=endtime)

In [8]:
df.head()

,,HorizontalWind,wspd,wdir,wdir_std,T,RH,P,SW_down,T10X,p10X
datetime,height,,,,,,,,,,
2016-11-21 17:00:00,3.0,0.871,0.858,271.5,9.780,279.481,84.2,959.58,255.8,6.605,14.17
2016-11-21 17:01:00,3.0,1.076,1.074,264.6,3.722,279.509,83.2,959.51,263.0,6.665,13.58
2016-11-21 17:02:00,3.0,1.204,1.192,260.4,7.930,279.552,83.0,959.51,264.6,6.732,13.37
2016-11-21 17:03:00,3.0,1.093,1.081,274.8,8.350,279.622,83.3,959.56,257.6,6.839,13.73
2016-11-21 17:04:00,3.0,1.552,1.545,260.3,5.421,279.703,83.5,959.51,268.8,6.907,13.69


In [9]:
df.tail()

,,HorizontalWind,wspd,wdir,wdir_std,T,RH,P,SW_down,T10X,p10X
datetime,height,,,,,,,,,,
2016-11-22 03:56:00,3.0,1.249,1.242,212.7,6.455,277.065,82.0,963.77,0.0,3.701,12.78
2016-11-22 03:57:00,3.0,0.699,0.688,234.8,10.360,276.980,80.3,963.78,0.0,3.672,12.78
2016-11-22 03:58:00,3.0,1.215,1.199,240.3,9.320,276.835,80.8,963.73,0.0,3.648,12.77
2016-11-22 03:59:00,3.0,1.405,1.395,220.1,6.738,276.844,82.2,963.75,0.0,3.623,12.78
2016-11-22 04:00:00,3.0,1.545,1.543,207.2,3.276,276.976,82.2,963.75,0.0,3.598,12.77


At this point, units and column names have been standardized according to the sensor data format specification.

Call `metmast.standard_output()` to reorder columns and (optionally) write output.

In [10]:
ps7 = metmast.standard_output(df)
ps7.head()

,,wspd,wdir,HorizontalWind,wdir_std,T,RH,P,SW_down,T10X,p10X
datetime,height,,,,,,,,,,
2016-11-21 17:00:00,3.0,0.858,271.5,0.871,9.780,279.481,84.2,959.58,255.8,6.605,14.17
2016-11-21 17:01:00,3.0,1.074,264.6,1.076,3.722,279.509,83.2,959.51,263.0,6.665,13.58
2016-11-21 17:02:00,3.0,1.192,260.4,1.204,7.930,279.552,83.0,959.51,264.6,6.732,13.37
2016-11-21 17:03:00,3.0,1.081,274.8,1.093,8.350,279.622,83.3,959.56,257.6,6.839,13.73
2016-11-21 17:04:00,3.0,1.545,260.3,1.552,5.421,279.703,83.5,959.51,268.8,6.907,13.69


In [11]:
# write out standardized timeseries data in csv format
metmast.standard_output(df,'test.csv')

In [12]:
# write out standardized timeseries data in netcdf format
metmast.standard_output(df,'test.nc')

## Temperature/pressure conversions
As an example, pick an arbitrary set of hourly mean values

In [13]:
hr_mean = ps7.xs(3,level='height').resample('1h').mean()
hr_mean

,wspd,wdir,HorizontalWind,wdir_std,T,RH,P,SW_down,T10X,p10X
datetime,,,,,,,,,,
2016-11-21 17:00:00,1.944483,240.593333,1.958867,6.601150,280.433667,80.503333,959.679833,292.691667,8.233717,13.519167
2016-11-21 18:00:00,2.844367,251.378333,2.864533,6.527967,280.612000,83.845000,960.026000,235.665000,9.661167,13.455833
2016-11-21 19:00:00,4.265933,251.836667,4.302117,7.274217,282.391000,78.631667,960.055167,382.946667,10.185167,13.479667
2016-11-21 20:00:00,5.820167,255.511667,5.859383,6.527083,282.636833,77.340000,960.099833,342.491667,11.307333,13.521000
2016-11-21 21:00:00,6.272000,251.678333,6.310000,6.242950,282.516333,75.923333,960.322667,242.920500,11.641833,13.466667
2016-11-21 22:00:00,6.104433,254.995000,6.141233,6.223867,282.975000,70.868333,960.645167,231.143333,11.465500,13.476333
2016-11-21 23:00:00,5.343033,257.641667,5.366133,5.240950,281.474667,72.718000,961.312000,68.300333,11.219333,13.331667
2016-11-22 00:00:00,2.927500,252.476667,2.936183,4.317467,279.763983,78.291667,962.066667,4.760350,8.786167,12.924167
2016-11-22 01:00:00,2.385250,230.681667,2.395933,4.835133,278.799167,80.931667,962.662333,0.000000,6.896683,12.847833


In [14]:
testdata = hr_mean.loc['2016-11-21 23:00']
testdata[['T','RH','P']]

T     281.474667
RH     72.718000
P     961.312000
Name: 2016-11-21 23:00:00, dtype: float64

### saturated vapor pressure <font color='green'>- PASSED</font>
https://www.weather.gov/epz/wxcalc_vaporpressure gives $e_s$ = 10.97 mb

In [15]:
fun.e_s(testdata['T'],model='Tetens') # Tetens' formula (default)

10.97058660841359

In [16]:
fun.e_s(testdata['T'],model='Bolton') # Bolton (1980), Mon. Weather Rev., Vol 108

10.961379766199544

### dewpoint temperature <font color='orange'>- nearly identical</font>
https://www.weather.gov/epz/wxcalc_rh gives $T_d$ = 276.86 K (and wet-bulb temp = 279.22 K)

In [17]:
fun.T_d(testdata['T'],testdata['RH'])

276.872367630902

### mixing ratio <font color='orange'>- nearly identical</font>
https://www.weather.gov/epz/wxcalc_mixingratio gives $w_s$ = 7.17 g/kg (= 0.00717 kg/kg)

In [18]:
fun.w_s(testdata['T'],testdata['P'])

0.007180266769687286

In [19]:
# sanity check, assuming p >> e_s (Wallace & Hobbs, Eqn. 3.63)
fun.epsilon * fun.e_s(testdata['T']) / testdata['P']

0.007098324862722254

### virtual temperature <font color='green'>- PASSED</font>
https://www.weather.gov/epz/wxcalc_virtualtemperature (using $T_d$ calculated from https://www.weather.gov/epz/wxcalc_rh) gives $T_v$ = 282.36

In [20]:
fun.T_to_Tv(testdata['T'], p=testdata['P'], RH=testdata['RH'])

282.36317505413456

In [21]:
Td = fun.T_d(testdata['T'],testdata['RH'])
fun.T_to_Tv(testdata['T'], Td=Td, p=testdata['P'])

282.3604000240343